In [2]:

!pip install transformers accelerate peft torch --quiet


In [3]:
#loading the model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "./distilgpt2-wekeza-finetuned_v3_lora"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D(nf=2304, nx=768)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (c_proj): lora.Linear(
            (base_layer): Conv1D(nf=768, nx=768)
            (l

In [4]:
#text generation function
def generate_response(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [5]:
#zer shot prompt
prompt_0shot = "Instruction: Summarize this week's Money Market Fund performance in Kenya.\nOutput:"
response_0shot = generate_response(prompt_0shot)
print("0-shot Response:\n", response_0shot)


0-shot Response:
 Instruction: Summarize this week's Money Market Fund performance in Kenya.
Output:
M-Pesa: 6.5% daily (8-11%)
### Response:
M-Pesa: 5.7% daily (8-11%)
### Response:
M-Pesa: 4.7% daily (8-11%)
### Response:
M-Pesa: 3.5% daily (8-11%)
### Response:
M-Pesa: 2.9% daily (8-11%)
### Response:


In [6]:
#I shot prompt
prompt_1shot = """Example:
Instruction: Summarize this week's performance of Cytonn Money Market Fund.
Output: Cytonn MMF delivered 11.2% annualized returns with stable liquidity and low risk.

Instruction: Summarize this week's Money Market Fund performance in Kenya.
Output:"""

response_1shot = generate_response(prompt_1shot)
print("1-shot Response:\n", response_1shot)


1-shot Response:
 Example:
Instruction: Summarize this week's performance of Cytonn Money Market Fund.
Output: Cytonn MMF delivered 11.2% annualized returns with stable liquidity and low risk.

Instruction: Summarize this week's Money Market Fund performance in Kenya.
Output: Cytonn MMF delivered 11.2% annualized returns with stable liquidity and low risk.
### Response: Cytonn MMF delivered 10.2% annualized returns with stable liquidity and low risk. Cytonn MMF delivered 9.2% annualized returns with stable liquidity and low risk. Cytonn MMF delivered 9.2% annualized returns with stable liquidity and low risk. Cytonn MMF delivered 9.2% annualized returns with stable liquidity


In [7]:
#few shot prompt
prompt_fewshot = """Example 1:
Instruction: Provide a report on Sanlam Money Market Fund.
Output: Sanlam MMF posted 10.9% returns this week, maintaining steady growth.

Example 2:
Instruction: Report on the performance of NCBA MMF.
Output: NCBA MMF offered 11.0% returns, highlighting investor confidence.

Instruction: Summarize Kenya's MMF performance this week.
Output:"""

response_fewshot = generate_response(prompt_fewshot)
print("Few-shot Response:\n", response_fewshot)


Few-shot Response:
 Example 1:
Instruction: Provide a report on Sanlam Money Market Fund.
Output: Sanlam MMF posted 10.9% returns this week, maintaining steady growth.

Example 2:
Instruction: Report on the performance of NCBA MMF.
Output: NCBA MMF offered 11.0% returns, highlighting investor confidence.

Instruction: Summarize Kenya's MMF performance this week.
Output: 1,000 returns.
### Response: Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report on performance of NCBA MMF. Report


In [8]:
#comparisons
print("=== 0-Shot ===")
print(response_0shot)
print("\n=== 1-Shot ===")
print(response_1shot)
print("\n=== Few-Shot ===")
print(response_fewshot)


=== 0-Shot ===
Instruction: Summarize this week's Money Market Fund performance in Kenya.
Output:
M-Pesa: 6.5% daily (8-11%)
### Response:
M-Pesa: 5.7% daily (8-11%)
### Response:
M-Pesa: 4.7% daily (8-11%)
### Response:
M-Pesa: 3.5% daily (8-11%)
### Response:
M-Pesa: 2.9% daily (8-11%)
### Response:

=== 1-Shot ===
Example:
Instruction: Summarize this week's performance of Cytonn Money Market Fund.
Output: Cytonn MMF delivered 11.2% annualized returns with stable liquidity and low risk.

Instruction: Summarize this week's Money Market Fund performance in Kenya.
Output: Cytonn MMF delivered 11.2% annualized returns with stable liquidity and low risk.
### Response: Cytonn MMF delivered 10.2% annualized returns with stable liquidity and low risk. Cytonn MMF delivered 9.2% annualized returns with stable liquidity and low risk. Cytonn MMF delivered 9.2% annualized returns with stable liquidity and low risk. Cytonn MMF delivered 9.2% annualized returns with stable liquidity

=== Few-Shot =